In [47]:
from utils import MysqlConnector
import pandas as pd
conn = MysqlConnector()

In [48]:
conn.execute('DROP TABLE IF EXISTS customer_stg')
conn.execute('''
CREATE TABLE customer_stg (
    customer_number INT,
    customer_name CHAR(30),
    customer_street_address CHAR(30),
    customer_zip_code INT(5),
    customer_city CHAR(30),
    customer_state CHAR(2)
);''')

In [49]:
df = pd.read_csv('assets/customer.csv')
df.head()

,customer_number,customer_name,customer_street_address,customer_zip_code,customer_city,customer_state
0,21,Really Large Customers,7500 Louise Dr.,17050,Mechanicsburg,PA
1,22,Small Stores,2500 Woodland St.,17055,Pittsburgh,PA
2,23,Medium Retailers,1111 Ritter Rd.,17055,Pittsburgh,PA
3,24,Good Companies,9500 Scott St.,17050,Mechanicsburg,PA
4,25,Wonderful Shops,3333 Rossmoyne Rd.,17050,Mechanicsburg,PA


In [50]:
conn.batch_insert('customer_stg', df.columns.tolist(), df.values.tolist())

In [51]:
conn.fetch('SELECT * FROM customer_stg;')[:2]

[(21,
  'Really Large Customers',
  '7500 Louise Dr.',
  17050,
  'Mechanicsburg',
  'PA'),
 (22, 'Small Stores', '2500 Woodland St.', 17055, 'Pittsburgh', 'PA')]

In [52]:
len(conn.fetch('SELECT * FROM customer_dim;'))

20

In [53]:
conn.execute('''
UPDATE customer_dim a, customer_stg b
SET a.customer_name = b.customer_name
WHERE     a.customer_number = b.customer_number
      AND a.expiry_date = '9999-12-31'
      AND a.customer_name <> b.customer_name
;''')

conn.execute('''INSERT INTO customer_dim
SELECT
  NULL	
, customer_number
, customer_name
, customer_street_address
, customer_zip_code
, customer_city
, customer_state
, CURRENT_DATE
, '9999-12-31'
FROM customer_stg 	
WHERE customer_number NOT IN(
SELECT b.customer_number 
FROM customer_dim a, customer_stg b 
WHERE a.customer_number = b.customer_number )
;
''')

In [54]:
len(conn.fetch('SELECT * FROM customer_dim;'))

40

In [55]:
conn.execute('DROP TABLE IF EXISTS product_stg')
conn.execute('''CREATE TABLE IF NOT EXISTS product_stg
( product_code INT
, product_name CHAR(30)
, product_category CHAR(30))
;''')

In [56]:
len(conn.fetch('SELECT * FROM product_stg;'))

0

In [57]:
queries = '''
INSERT INTO product_stg VALUES (1,'Hard Disk','Storage');
INSERT INTO product_stg VALUES (2,'Pen Drive','Storage');
INSERT INTO product_stg VALUES (3,'LCD Panel','Monitor');
INSERT INTO product_stg VALUES (4,'Keyboard','Peripheral');
INSERT INTO product_stg VALUES (5,'Mouse','Peripheral');
INSERT INTO product_stg VALUES (6,'Filter','Monitor');
INSERT INTO product_stg VALUES (7,'Batery','Peripheral');
INSERT INTO product_stg VALUES (8,'Print Cable','Peripheral');
INSERT INTO product_stg VALUES (9,'Ink Print','Peripheral');
INSERT INTO product_stg VALUES (10,'WebCam','Monitor');
INSERT INTO product_stg VALUES (11,'Microphone','Peripheral');
INSERT INTO product_stg VALUES (12,'Pocket Storage','Storage');
INSERT INTO product_stg VALUES (13,'Backup Storage','Storage');
INSERT INTO product_stg VALUES (14,'External Cam','Monitor');
INSERT INTO product_stg VALUES (15,'Head Phone ','Peripheral');
'''.split('\n')

for query in queries:
    conn.execute(query)

In [58]:
len(conn.fetch('SELECT * FROM product_stg;'))

15

In [59]:
len(conn.fetch('SELECT * FROM product_dim'))

0

In [60]:
conn.execute('''
UPDATE 
  product_dim a
, product_stg b
SET
  expiry_date = SUBDATE(CURRENT_DATE, 1)
WHERE
    a.product_code = b.product_code
AND ( a.product_name <> b.product_name OR a.product_category <> b.product_category )
AND expiry_date = '9999-12-31'
;''')

conn.execute('''INSERT INTO product_dim 
SELECT
  NULL	
, b.product_code
, b.product_name
, b.product_category
, CURRENT_DATE
, '9999-12-31'
FROM 
  product_dim a
, product_stg b	
WHERE 
    a.product_code = b.product_code
AND (   a.product_name <> b.product_name
     OR a.product_category <> b.product_category )
AND EXISTS 
( SELECT * FROM product_dim x 
  WHERE   b.product_code = x.product_code 
      AND a.expiry_date = SUBDATE(CURRENT_DATE, 1) )
AND NOT EXISTS 
( SELECT *
  FROM product_dim y 
  WHERE     b.product_code = y.product_code 
        AND y.expiry_date = '9999-12-31' )
;''')

conn.execute('''INSERT INTO product_dim
SELECT
  NULL	
, product_code
, product_name
, product_category
, CURRENT_DATE
, '9999-12-31'
FROM product_stg 	
WHERE product_code NOT IN(
SELECT y.product_code 
FROM product_dim x, product_stg y 
WHERE x.product_code = y.product_code )
;

''')

In [61]:
len(conn.fetch('SELECT * FROM product_dim'))

15

In [62]:
print(f"Orders: {len(conn.fetch('SELECT * FROM order_dim;'))}")
print(f"Dates: {len(conn.fetch('SELECT * FROM date_dim;'))}")
print(f"Sales: {len(conn.fetch('SELECT * FROM sales_order_fact;'))}")

Orders: 0
Dates: 0
Sales: 0


In [63]:
conn.execute('''
INSERT INTO order_dim VALUES
  (NULL, 1, CURRENT_DATE, '9999-12-31')
, (NULL, 2, CURRENT_DATE, '9999-12-31')
, (NULL, 3, CURRENT_DATE, '9999-12-31')
, (NULL, 4, CURRENT_DATE, '9999-12-31')
, (NULL, 5, CURRENT_DATE, '9999-12-31')
, (NULL, 6, CURRENT_DATE, '9999-12-31')
, (NULL, 7, CURRENT_DATE, '9999-12-31')
, (NULL, 8, CURRENT_DATE, '9999-12-31')
, (NULL, 9, CURRENT_DATE, '9999-12-31')
, (NULL, 10, CURRENT_DATE, '9999-12-31')
, (NULL, 11, CURRENT_DATE, '9999-12-31')
, (NULL, 12, CURRENT_DATE, '9999-12-31')
, (NULL, 13, CURRENT_DATE, '9999-12-31')
, (NULL, 14, CURRENT_DATE, '9999-12-31')
, (NULL, 15, CURRENT_DATE, '9999-12-31')
, (NULL, 16, CURRENT_DATE, '9999-12-31')
, (NULL, 17, CURRENT_DATE, '9999-12-31')
, (NULL, 18, CURRENT_DATE, '9999-12-31')
, (NULL, 19, CURRENT_DATE, '9999-12-31')
, (NULL, 20, CURRENT_DATE, '9999-12-31')

;''')

conn.execute('''INSERT INTO date_dim VALUES
  (NULL, '2005-01-31', 'January' , 1, 1, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-02-28', 'February', 2, 1, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-03-31', 'March', 3, 1, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-04-30', 'April' , 4, 2, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-05-31', 'May', 5, 2, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-06-30', 'June', 6, 2, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-07-31', 'July' , 7, 3, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-08-31', 'August', 8, 3, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-09-30', 'September', 9, 3, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-10-31', 'October' , 10, 4, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-11-30', 'November', 11, 4, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2005-12-31', 'December', 12, 4, 2005, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-01-31', 'January' , 1, 1, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-02-28', 'February', 2, 1, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-03-31', 'March', 3, 1, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-04-30', 'April' , 4, 2, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-05-31', 'May', 5, 2, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-06-30', 'June', 6, 2, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-07-31', 'July' , 7, 3, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-08-31', 'August', 8, 3, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-09-30', 'September', 9, 3, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-10-31', 'October' , 10, 4, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-11-30', 'November', 11, 4, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2006-12-31', 'December', 12, 4, 2006, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-01-31', 'January' , 1, 1, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-02-28', 'February', 2, 1, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-03-31', 'March', 3, 1, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-04-30', 'April' , 4, 2, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-05-31', 'May', 5, 2, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-06-30', 'June', 6, 2, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-07-31', 'July' , 7, 3, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-08-31', 'August', 8, 3, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-09-30', 'September', 9, 3, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-10-31', 'October' , 10, 4, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-11-30', 'November', 11, 4, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2007-12-31', 'December', 12, 4, 2007, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-01-31', 'January' , 1, 1, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-02-28', 'February', 2, 1, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-03-31', 'March', 3, 1, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-04-30', 'April' , 4, 2, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-05-31', 'May', 5, 2, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-06-30', 'June', 6, 2, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-07-31', 'July' , 7, 3, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-08-31', 'August', 8, 3, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-09-30', 'September', 9, 3, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-10-31', 'October' , 10, 4, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-11-30', 'November', 11, 4, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2008-12-31', 'December', 12, 4, 2008, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-01-31', 'January' , 1, 1, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-02-28', 'February', 2, 1, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-03-31', 'March', 3, 1, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-04-30', 'April' , 4, 2, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-05-31', 'May', 5, 2, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-06-30', 'June', 6, 2, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-07-31', 'July' , 7, 3, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-08-31', 'August', 8, 3, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-09-30', 'September', 9, 3, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-10-31', 'October' , 10, 4, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-11-30', 'November', 11, 4, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2009-12-31', 'December', 12, 4, 2009, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-01-31', 'January' , 1, 1, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-02-28', 'February', 2, 1, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-03-31', 'March', 3, 1, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-04-30', 'April' , 4, 2, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-05-31', 'May', 5, 2, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-06-30', 'June', 6, 2, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-07-31', 'July' , 7, 3, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-08-31', 'August', 8, 3, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-09-30', 'September', 9, 3, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-10-31', 'October' , 10, 4, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-11-30', 'November', 11, 4, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2010-12-31', 'December', 12, 4, 2010, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-01-31', 'January' , 1, 1, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-02-28', 'February', 2, 1, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-03-31', 'March', 3, 1, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-04-30', 'April' , 4, 2, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-05-31', 'May', 5, 2, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-06-30', 'June', 6, 2, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-07-31', 'July' , 7, 3, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-08-31', 'August', 8, 3, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-09-30', 'September', 9, 3, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-10-31', 'October' , 10, 4, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-11-30', 'November', 11, 4, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2011-12-31', 'December', 12, 4, 2011, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-01-31', 'January' , 1, 1, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-02-28', 'February', 2, 1, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-03-31', 'March', 3, 1, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-04-30', 'April' , 4, 2, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-05-31', 'May', 5, 2, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-06-30', 'June', 6, 2, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-07-31', 'July' , 7, 3, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-08-31', 'August', 8, 3, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-09-30', 'September', 9, 3, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-10-31', 'October' , 10, 4, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-11-30', 'November', 11, 4, 2012, CURRENT_DATE, '9999-12-31')
, (NULL, '2012-12-31', 'December', 12, 4, 2012, CURRENT_DATE, '9999-12-31')
;''')


conn.execute('''INSERT INTO sales_order_fact VALUES
  (1, 1, 12, 1, 1000)
, (2, 2, 3, 2, 1000)
, (3, 3, 7, 3, 4000)
, (4, 4, 2, 4, 4000)
, (5, 5, 5, 5, 6000)
, (6, 1, 4, 6, 6000)
, (7, 2, 12, 7, 8000)
, (8, 3, 3, 8, 8000)
, (9, 4, 5, 9, 10000)
, (10, 6, 2, 10, 1200)
, (11, 6, 5, 11, 4400)
, (12, 7, 9, 12, 1500)
, (13, 8, 4, 13, 4340)
, (14, 8, 2, 14, 7600)
, (15, 8, 7, 15, 6000)
, (16, 9, 4, 16, 5000)
, (17, 10, 2, 17, 6000)
, (18, 10, 7, 18, 8500)
, (19, 9, 4, 19, 10500)
, (20, 8, 12, 20, 16000)
, (21, 6, 7, 21, 15500)
, (22, 7, 3, 22, 15200)
, (23, 2, 4, 23, 14440)
, (24, 3, 2, 24, 20600)
, (25, 5, 14, 25, 21000)
, (26, 5, 7, 26, 4000)
, (27, 6, 5, 27, 2000)
, (28, 7, 5, 28, 1500)
, (29, 9, 10, 29, 2500)
, (30, 8, 10, 30, 3000)
, (31, 1, 7, 31, 15500)
, (32, 1, 3, 32, 1200)
, (33, 2, 4, 33, 13440)
, (34, 3, 2, 34, 26400)
, (35, 4, 4, 35, 2500)
, (36, 4, 7, 36, 4600)
, (37, 6, 5, 37, 2700)
, (38, 7, 5, 38, 1700)
, (39, 9, 10, 39, 2700)
, (40, 8, 10, 40, 3700)
, (41, 1, 7, 41, 15500)
, (42, 1, 3, 42, 1200)
, (43, 2, 4, 43, 13440)
, (44, 3, 5, 44, 400)
, (45, 4, 6, 45, 1230)
, (46, 4, 1, 46, 640)
, (47, 6, 5, 47, 550)
, (48, 7, 5, 48, 1500)
, (49, 9, 10, 49, 1700)
, (50, 8, 10, 50, 2700)
, (51, 1, 12, 51, 1000)
, (52, 2, 13, 52, 1000)
, (53, 3, 7, 53, 4000)
, (54, 4, 12, 54, 4000)
, (55, 5, 15, 55, 6000)
, (56, 1, 4, 56, 6000)
, (57, 2, 2, 57, 8000)
, (58, 3, 3, 58, 8000)
, (59, 4, 15, 59, 10000)
, (60, 6, 2, 60, 1200)
, (61, 6, 15, 61, 4400)
, (62, 7, 3, 62, 1500)
, (63, 8, 14, 63, 4340)
, (64, 8, 2, 64, 7600)
, (65, 8, 7, 65, 6000)
, (66, 9, 14, 66, 5000)
, (67, 10, 12, 67, 6000)
, (68, 10, 7, 68, 8500)
, (69, 9, 4, 69, 10500)
, (70, 8, 2, 70, 16000)
, (71, 1, 2, 1, 1000)
, (72, 2, 13, 72, 1000)
, (73, 3, 7, 73, 4000)
, (74, 4, 2, 74, 4000)
, (75, 5, 5, 5, 6000)
, (76, 1, 4, 16, 6000)
, (77, 2, 2, 17, 8000)
, (78, 3, 3, 18, 8000)
, (79, 4, 5, 19, 10000)
, (80, 6, 2, 80, 1200)
, (81, 6, 5, 81, 4400)
, (82, 7, 3, 82, 1500)
, (83, 8, 4, 83, 4340)
, (84, 8, 2, 84, 7600)
, (85, 8, 7, 85, 6000)
, (86, 9, 4, 86, 5000)
, (87, 10, 11, 87, 6000)
, (88, 10, 7, 88, 8500)
, (89, 9, 11, 89, 10500)
, (90, 8, 2, 90, 16000)
, (91, 1, 12, 91, 1000)
, (92, 2, 3, 92, 1000)
, (93, 3, 7, 93, 4000)
, (94, 4, 2, 94, 4000)
, (95, 5, 5, 95, 6000)
, (96, 1, 4, 96, 6000)
, (97, 2, 2, 37, 8000)
, (98, 3, 13, 38, 8000)
, (99, 4, 5, 39, 10000)
, (100, 6, 12, 60, 1200)
, (101, 8, 2, 60, 16000)
, (102, 1, 12, 6, 1000)
, (103, 2, 3, 62, 1000)
, (104, 3, 11, 63, 4000)
, (105, 4, 2, 54, 4000)
, (106, 5, 10, 55, 6000)
, (107, 1, 4, 56, 6000)
, (108, 2, 2, 67, 8000)
, (109, 3, 3, 68, 8000)
, (110, 4, 5, 39, 10000)
, (111, 8, 2, 10, 16000)
, (112, 1, 2, 31, 1000)
, (113, 2, 3, 22, 1000)
, (114, 3, 12, 23, 4000)
, (115, 4, 2, 54, 4000)
, (116, 5, 5, 75, 6000)
, (117, 1, 4, 76, 6000)
, (118, 2, 9, 87, 8000)
, (119, 3, 3, 90, 8000)
, (120, 4, 5, 9, 10000)
, (121, 11, 2, 11, 16000)
, (122, 11, 9, 12, 1000)
, (123, 12, 9, 21, 1000)
, (124, 13, 7, 32, 4000)
, (125, 14, 2, 43, 4000)
, (126, 15, 5, 54, 6000)
, (127, 11, 4, 65, 6000)
, (128, 12, 2, 76, 8000)
, (129, 13, 3, 86, 8000)
, (130, 14, 5, 9, 10000)
, (131, 15, 9, 30, 16000)
, (132, 16, 2, 13, 1000)
, (133, 16, 3, 25, 1000)
, (134, 17, 7, 35, 4000)
, (135, 19, 2, 45, 4000)
, (136, 21, 9, 55, 6000)
, (137, 20, 4, 63, 6000)
, (138, 22, 9, 73, 8000)
, (139, 24, 3, 82, 8000)
, (140, 23, 5, 90, 10000)
, (141, 25, 9, 91, 16000)
, (142, 26, 12, 1, 1000)
, (143, 36, 13, 2, 1000)
, (144, 37, 7, 34, 4000)
, (145, 40, 11, 45, 4000)
, (146, 39, 5, 56, 6000)
, (147, 38, 4, 67, 6000)
, (148, 33, 9, 77, 8000)
, (149, 32, 9, 83, 8000)
, (150, 31, 5, 93, 10000)
, (151, 28, 12, 10, 16000)
, (152, 27, 2, 1, 1000)
, (153, 26, 3, 22, 1000)
, (154, 27, 7, 32, 4000)
, (155, 22, 2, 44, 4000)
, (156, 38, 5, 52, 6000)
, (157, 32, 14, 16, 6000)
, (158, 31, 12, 27, 8000)
, (159, 30, 13, 83, 8000)
, (160, 32, 9, 19, 10000)
, (161, 35, 11, 13, 16000)
, (162, 16, 2, 15, 1000)
, (163, 26, 3, 25, 1000)
, (164, 37, 15, 23, 4000)
, (165, 39, 16, 24, 4000)
, (166, 31, 10, 25, 6000)
, (167, 40, 14, 46, 6000)
, (168, 32, 2, 57, 8000)
, (169, 22, 3, 68, 8000)
, (170, 25, 5, 79, 10000)
, (171, 35, 2, 81, 16000)
, (172, 36, 12, 88, 1000)
, (173, 40, 7, 86, 1000)
, (174, 40, 11, 84, 4000)
, (175, 40, 2, 85, 4000)
, (176, 35, 5, 89, 6000)
, (177, 38, 4, 56, 6000)
, (178, 39, 12, 57, 8000)
, (179, 38, 13, 68, 8000)
, (180, 37, 5, 69, 10000)
, (181, 35, 12, 70, 16000)
, (182, 26, 2, 71, 1000)
, (183, 40, 3, 82, 1000)
, (184, 37, 11, 83, 4000)
, (185, 31, 2, 74, 4000)
, (186, 32, 5, 56, 6000)
, (187, 25, 4, 66, 6000)
, (188, 28, 2, 76, 8000)
, (189, 29, 13, 68, 8000)
, (190, 30, 5, 59, 10000)
, (191, 35, 2, 10, 16000)
, (192, 26, 2, 16, 1000)
, (193, 40, 3, 26, 1000)
, (194, 27, 10, 73, 4000)
, (195, 12, 12, 44, 4000)
, (196, 21, 5, 45, 6000)
, (197, 24, 14, 6, 6000)
, (198, 32, 12, 7, 8000)
, (199, 33, 3, 48, 8000)
, (200, 35, 15, 39, 10000)
;
''')

In [64]:
print(f"Orders: {len(conn.fetch('SELECT * FROM order_dim;'))}")
print(f"Dates: {len(conn.fetch('SELECT * FROM date_dim;'))}")
print(f"Sales: {len(conn.fetch('SELECT * FROM sales_order_fact;'))}")

Orders: 20
Dates: 96
Sales: 200
